<a href="https://colab.research.google.com/github/BabritB/neural-networks/blob/master/Summarization_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
! mkdir  ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
! chmod 600 ~/.kaggle/kaggle.json


In [0]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

In [0]:
from zipfile import ZipFile
file_name = 'amazon-fine-food-reviews.zip'
with ZipFile(file_name,'r') as zip:
   zip.extractall()
print('Done')

In [0]:
import pandas as pd
Data = pd.read_csv('Reviews.csv')
print(Data.head())

In [0]:
Data.columns

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential,Model
# from keras.layers import Dense,Embedding,LSTM,Bidirectional,Dropout,Concatenate,TimeDistributed,Input

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [0]:
data=pd.read_csv('Reviews.csv')

In [0]:
data.drop_duplicates(subset=['Text'],inplace=True)

In [0]:
data.dropna(axis=0,inplace=True)

In [0]:
# Preprocessing:
#Contraction dictionary:

In [0]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [0]:
#Presprocessing Task:
# Convert everything to lowercase
# Remove HTML tags
# Contraction mapping
# Remove (‘s)
# Remove any text inside the parenthesis ( )
# Eliminate punctuations and special characters
# Remove stopwords
# Remove short word
nltk.download('stopwords')
STOP_WORDS=stopwords.words('english')

In [0]:
HTML_RE=re.compile(r'<[^>]+>')
STOP_WORDS=stopwords.words('english')
def text_clean_helper(text):
    new_txt=text.lower()
    new_txt=HTML_RE.sub('',new_txt)
    new_txt=' '.join([contraction_mapping[txt] if txt in contraction_mapping else txt for txt in new_txt.split()])
    new_txt=re.sub(r"'s\b","",new_txt)
    new_txt=re.sub("[^a-zA-Z]", " ", new_txt)
    new_txt=' '.join([word.strip() for word in new_txt.split() if word not in STOP_WORDS])
    return new_txt

In [0]:
data['Text']=data['Text'].apply(lambda x:text_clean_helper(x))
data['Summary']=data['Summary'].apply(lambda x:text_clean_helper(x))

In [0]:
# Add the START and END tag in Summary as it will be the target:
data['Summary']=data['Summary'].apply(lambda x:'<START> '+x+' <END>')

In [0]:
# Length distribution of Text and Summary:
text_len=[]
summary_len=[]
for sent in data['Text']:
    text_len.append(len(sent.split()))
for sent in data['Summary']:
    summary_len.append(len(sent.split()))

In [0]:
length_df=pd.DataFrame({'Text':text_len,'Summary':summary_len})
style.use('fivethirtyeight')
sns.distplot(length_df['Text'])

In [0]:
style.use('fivethirtyeight')
sns.distplot(length_df['Summary'])

In [0]:
# max len majority in Text is 80 and for summary 10:
max_len_text=80 
max_len_summary=10

In [0]:
# Train test split:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(np.array(data['Text']),np.array(data['Summary']),test_size=0.1,random_state=0,shuffle=True)

In [0]:
# Preparing Tokenizers for both Text and Summary:
text_tokenizer=Tokenizer()
text_tokenizer.fit_on_texts(list(X_train))

In [0]:
X_train_seq=text_tokenizer.texts_to_sequences(X_train)
X_test_seq=text_tokenizer.texts_to_sequences(X_test)

X_train_pad=pad_sequences(X_train_seq,maxlen=max_len_text,padding='post')
X_test_pad=pad_sequences(X_test_seq,maxlen=max_len_text,padding='post')

In [0]:
sum_tokenizer=Tokenizer()
sum_tokenizer.fit_on_texts(list(y_train))
y_train_seq=sum_tokenizer.texts_to_sequences(y_train)
y_test_seq=sum_tokenizer.texts_to_sequences(y_test)

y_train_pad=pad_sequences(y_train_seq,maxlen=max_len_summary,padding='post')
y_test_pad=pad_sequences(y_test_seq,maxlen=max_len_summary,padding='post')

In [0]:
VOCAB_SIZE_SUM=len(sum_tokenizer.word_index)+1
y_test_pad.shape
VOCAB_SIZE_TXT=len(text_tokenizer.word_index)+1
VOCAB_SIZE_TXT

## Model Building:

In [0]:
from attention import AttentionLayer
from keras import backend as K 
K.clear_session()

latent_dim = 300
embedding_dim=100

# Encoder
encoder_inputs = Input(shape=(max_len_text,))

#embedding layer
enc_emb =  Embedding(VOCAB_SIZE_TXT, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(VOCAB_SIZE_SUM, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(VOCAB_SIZE_SUM, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

In [0]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [0]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

In [0]:
history=model.fit([X_train_pad,y_train_pad[:,:-1]], y_train_pad.reshape(y_train_pad.shape[0],y_train_pad.shape[1], 1)[:,1:] ,
                  epochs=10,batch_size=128, 
                  validation_data=([X_test_pad,y_test_pad[:,:-1]],
                  y_test_pad.reshape(y_test_pad.shape[0],y_test_pad.shape[1], 1)[:,1:])
                 )